# Introducción

La detección de spam es una de las aplicaciones más conocidas de Machine Learning para la web. Casi todos los proveedores de servicios de correo tienen sistemas de detección de spam construidos y automaticamente clasifican el correo como Spam.

En este proyecto vamos a utilizar el algoritmo de Nave Bayes para crear un modelo que pueda clasificar un dataset de mensajes SMS en inglés como spam o no spam basado en la información de entrenamiento con la que alimentemos al modelo. Antes de iniciar, es importante tener cierto nivel de intuición de cómo un mensaje de texto de spam se pueda ver.

**¿Qué son mensajes de spam?**
Normalmente, palabras como 'free', 'win, 'cash', 'prize y similares indican que un mensaje es de spam. Están diseñados para llamar la atención y tentarte a abrirlos. Adicionalmente, algunos mensajes de spam usan sólo letras mayúsculas y muchos signos de exclamación. Normalmente, es muy fácil identificar para un humano si un mensaje de texto es spam, pero vamos a construir un modelo que lo haga. Este es un problema de clasificación binaria - los mensajes pueden ser 'Spam' o 'No Spam'. Utilizamos el siguiente dataset:

https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

## Paso 0. Naive Bayes

El Teorema de Bayes es uno de los primeros algoritmos de inferencia probabilistica. Fue propuesta por el Reverendo Bayes con el objetivo de inferir la existencia de Dios. Hasta el momento, este algoritmo funciona muy bien.

Digamos que eres un miembro del Servicio Secreto y te han pedido que cuides a un candidatos presidencial durante su campaña. Siendo eventos públicos y abiertos a todos, tu trabajo no es fácil y necesitas estar constantemente atento ante amenazas. Una manera de iniciar es poniendo un factor de amenaza para cada persona. Podemos decidir si una persona es una amenaza viable a parti de ciertos features como su edad, sexo, y otros factores como si la persona carra una mochila o se ve nervioa. 

Si un individuo da check a todos los factores de amenaza a un nivel que cruza cierto threshold o límite, puedes tomar acciones para alejar a la persona. El Teorema de Bayes funciona en la misma manera: programamos la probabilidad de un evento (una persona es una amenaza) a partir de las probabilidades de ciertos eventos relacionados como la edad, el sexo, la presencia de una mochila o no, si una persona está nerviosa, etc.

Una cosa que es importante considerar es que los features son independientes entre ellos. Por ejemplo, si un niño se ve nervioso en un evento, la probabilidad que sea una amenaza no es la misma que si es un hombre adulto nervioso. Considerar todos los features independientes de los otros es la parte 'naive' del algoritmo. No siempre todos los features son independientes y esto puede afectar la decisión final. Tal vez el sexo y el uso de bolso sean dependientes.

Resumiendo, el Teorema de Bayes calcula la probabilidad de un evento ocurriendo (un mensaje siendo spam) basado en las distribuciones probabilisticas unidas de ciertos eventos. Más adelante veremos más a detalle qué significa todo esto, pero vamos a enfocarnos en comprender la información por ahora.

## Paso 1. Comprender la información

Como mencionamos antes, vamos a utilizar un dataset del repositorio del UCI Machine Learning, el cuál tiene una muy buena colección de datasets para objetivos de investigación. 

![imagen de spam](https://github.com/udacity/machine-learning/raw/85dfd1dba5ccfd5435f7fb19a6274b2f95a6bfb7/projects/practice_projects/naive_bayes_tutorial/images/dqnb.png "Dataset")

Hay dos columnas en el dataset. Estas columnas no tienen ningún nombre por ahora. La primera columna toma dos valores, 'ham', que significa que un mensaje no es spam, y 'spam', que significa que el mensaje es, por sorpresa, spam. La segunda columna contiene el contenido del mensaje SMS como texto. 

Vamos a importar esta información para que la podamos utilizar bien:
* Importa el dataset utilizando Pandas. Utiliza el método ```read_table```. 
* Este dataset está separado con tabs, los cuales suelen ser representados como un `\t`. Utiliza el argumento ```sep``` para indicar esto.
* Utiliza el argumento `names` para indicar los nombres de las columnas (label y sms_message).
* Utiliza el método `head()` del dataframe para imprimir los primeros 5 registros de la tabla. 

In [ ]:
import pandas as pd
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df = #TODO

# Output printing out first 5 columns
df.head()

## Paso 2. Preprocesamos información

Ahora que ya tenemos un entendimiento básico de cómo se ve nuestro dataset, vamos a convertir nuestros labels a variables binarias. 0 para representar 'ham' y 1 para representar 'spam'. Esto facilitará la computación. Esto es importante para que scikit-learn pueda recibir el input: sólo acepta valores numéricos. Si enviamos strings (cadena de caracteres), scikit learn lo convertiría pero haría cosas raras y podría causar problemas cuando calculemos la precisión o el recall. Para evitar problemas no esperados, es buena práctica convertir cualquier valor categórico a enteros. 

Utiliza ```df.label.map``` para cambiar la columna label. En el mapa, utiliza `{'ham':0, 'spam':1}` como argumento para convertir ham a 0 y spam a 1.

In [ ]:
df['label'] = df.label.map(#TODO)
print(df.shape)
df.head()

## Paso 3.1 Bag of words

Lo que tenemos ahora es una larga colección de casi 6mil filas de data. La mayoría de los algoritmos de ML, como mencionamos, se basan en data numérico. Los correos y mensajes suelen ser pesados en texto. 

Vamos a introducir un nuevo concepto llamado Bag of Words (BoW). Es un término utilizado para colecciones de data de texto con cierta representación. La idea de BoW es tomar piezas del texto y contar la frecuencia de las palabras en ese texto. Es importante notar que en BoW, las palabras son tratadas independientemente y por lo tanto el contexto u orden de las palabras no importará.

¿Cómo hacemos esto? Ahora revisaremos el proceso. La idea, en general, es convertir la colección de documentos (en este caso, mensajes de texto) en una matriz. En esta matriz,cada fila corresponderá a un documento y cada columna a una palabra (token). El valor en una celda corresponde a la frecuencia de la palabra de la columna en el documento de la fila. Por ejemplo:

Digamos que tenemos 4 documentos:
```
['Hello, how are you!',
'Win money, win from home.',
'Call me now',
'Hello, Call you tomorrow?']
```

Nuestro objetivo es convertir ese conjunto de texto en la matriz de distribución de frecuencias

![matriz de bag of words](https://github.com/udacity/machine-learning/raw/85dfd1dba5ccfd5435f7fb19a6274b2f95a6bfb7/projects/practice_projects/naive_bayes_tutorial/images/countvectorizer.png "Matriz de Bag of Words")

Aquí, como puedes ver, los documentos están numerados en las filas, y cada palabra es el nombre de la columna. Vamos a ver cómo podemos hacer con un set pequeño de documento. Vamos a utilizar sklearns algo llamado count vectorizer. Este es un método que:
* Hace un token de un string. Es decir, convierte una oración en palabras.
* Asigna un ID entero para cada token.
* Cuenta la ocurrencia de cada token.

Puedes revisar la documentación en: 
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer

Notas
* El método CountVectorizer automáticamente convierte todas las palabras tokenizadas a minúsculas para que no trate palabras como 'Hola' y 'hola' de manera diferente. Esto se puede hacer con el parámetro ```lowercase``` que es True por default.

* También ignora toda la puntuación de las palabras seguidas por puntuación (Hola!). Esto hace que no sean tratadas como palabras diferentes. Esto lo logra utilizando el parámetro ```token_parameter```. El default de este parámetro acepta tokens con 2 o más caracteres alfanuméricos.

* El tercer parámetro mportante es `stop_words`. Stop words se refiere a las palabras más comunes en un lenguaje. Incluye palabras en inglés como 'am', 'an', 'and', 'the', etc. Al configurar este parámetro a inglés, `CountVectorizer` automáticamente ignorará todas las palabras comunes que se encuentren en la lista de palabras. Esto ayuda muchísimo porque facilitará que detectemos palabras indicativas de spam.

Suena a mucho, así que, antes de programar esto con scikit learn, vamos a programarlo por nuestra cuenta.

## Paso 3.2 Implementando BoW desde cero

### Paso 1. Convertir todas las palabras a minúsculas. 

Supongamos que tenemos el siguiente documento:
```
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
```
Convierte todas las palabras de estos documentos a sus versiones en minúsculas. Guárdalo en una lista llamda `lower_case_documents`. Puedes convertir todos los strings a minúscula utilizando el método ```lower```.

In [ ]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = []
for i in documents:
    #TODO
print(lower_case_documents)

### Paso 2. Eliminar puntuación

Elimina toda la puntuación de los strings del conjunto de documentos. Guárdalo en una lista llamada `sans_punctuation_documents`. Esto lo puedes hacer una línea, pero busca bien.

En la librería string, hay algo llamado string.punctuation. Revisa la documentación.

Puedes revisar esta discusión en stackoverflow
https://stackoverflow.com/questions/16050952/how-to-remove-all-the-punctuation-in-a-string-python

Sugiero la propuesta de la manera más eficiente. 

In [ ]:
sans_punctuation_documents = []
import string

for i in lower_case_documents:
   # TODO
    
print(sans_punctuation_documents)

### Paso 3. Tokenization
Tokenizar una oración significa dividir el documento en palabras individuales utilizando un delimitador. El delimitador especifica el caracter que utilizaremos para identificar el inicio y final de una palabra (para una oración, el delimitador suele ser un espacio). 

Tokeniza los strings guardados en `sans_punctuation_documents` utilizando el método `split()` de Python. Guarda el documento final en una lista llamada `preprocessed_documents`. Busca documentación de Python o busca en Google cuando tengas dudas.

In [ ]:
preprocessed_documents = []
for i in sans_punctuation_documents:
    #TODO
print(preprocessed_documents)

### Paso 4. Frecuencias

Ahora que ya tenemos nuestro conjunto de documentos en el formato requerido, podemos contar la ocurrencia de cada palabra en cada documento del conjunto. Utilizamores el método `Counter`de la librería `collections` de Python.

`Counter` cuentra cada ocurrencia de cada objeto (palabra) en una lista y devuelve un diccionario donde las llaves son el objeto y el valor correspondiente es la cuenta de ese objeto en la lista. Si no tiene mucho sentido esto, revisa:
* Diccionarios en Python: https://www.tutorialspoint.com/python/python_dictionary.htm
* Counter: https://pymotw.com/2/collections/counter.html

Utilizando el método `Counter()` y `preprocessed_documents` como entrada, crea un diccionario donde las llaves sean cada palabra en cada documento, y sus valores correspondientes sean la frecuencia de las ocurrencias de esa palabra. Guarda el diccionario en un objeto llamado `frequency_list`.

In [ ]:
import pprint
from collections import Counter

frequency_list = []

for i in preprocessed_documents:
    
pprint.pprint(frequency_list)

¡Excelente! Ya implementaste Bag of Words desde cero. Como puedes ver en el output anterior, tenemos un diccionario con la distribución de frecuencias que claramente dice cómo es el texto con el que lidiamos. 

Ahora que ya sabemos qué está pasando internamente, vamos a hacerlo con el método `sklearn.feature_extraction.text.CountVectorizer`.

## Paso 3.3 Implementando BoW con scikit-learn

Haremos lo mismo pero utilizaremos `scikit-learn` para hacerlo de una manera más concisa. No hay que reinventar la rueda. 

https://blog.codinghorror.com/dont-reinvent-the-wheel-unless-you-plan-on-learning-more-about-wheels/

In [ ]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

Importa el método de scikit learn y crea una instancia de `CountVectorizer`.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = #TODO

### Preprocesamiento de data con CountVectorizer()

En el paso 3.2, implementamos una versión de `CountVectorizer()` que limpiaba la información primero. Este paso de limpiar la información involucraba convertir todo a minúsculas y eliminar signos de puntuación. `CountVectorizer()` tiene ciertos parámetros que pueden encargarse de este paso. Estos son:

* `lowercase = True`

Este parámetro tiene un valor por default de True. Convierte todo a minúsculas.

* `token_pattern = (?u)\\b\\w\\w+\\b`

Esto se ve raro. Esto se llama expresión regular. Lo que hace es recibir el input e ignorar signos de puntuación y tratarlos como delimitadores, mientras que acepta cualquier string alfanumérico con una longitud igual o mayor a 2. Las expresiones regulares son muy útiles para todo lo de data science y analysis relacionado con texto. 

Te sugiero hacer el siguiente tutorial para aprender más del tema: https://regexone.com/

* `stop_words`

Este parámetro, si se le asigna inglés, borrará todas las palabras de los documentos que estén en una lista predefinida en scikit-learn. Considerando el tamaño de nuestro dataset y el hecho que manejamos mensajes SMS y no textos grandes como correos, no vamos a configurar este parámetro. 

Puedes ver los valores de todos los parámetros de nuestro objeto `count_vector` así:

In [ ]:
'''
Practice node:
Print the 'count_vector' object which is an instance of 'CountVectorizer()'
'''
print(count_vector)

Hacemos un fit de tu dataset con el objeto de CountVectorizer usando su método `fit()` y conseguimos la lista de palabras que han sido categorizadas como features usando el método `get_features_names()`.

In [ ]:
count_vector.fit(documents)
count_vector.get_feature_names()

El método `get_feature_names()` devuelve los nombres de los features para este dataset, que es el conjunto de palabras de nuestro vocabulario para `documents` (El vocabulario son las palabras que existen en un cuerpo de texto. No tiene sentido considerar "house" como parte del vocabulario si no existe en ningún punto de nuestra información). 

Crea una matriz donde las filas sean cada uno de los 4 documentos y las columnas sean cada palabras. El valor correspondiente al par (fila, columna) es la frecuencia de la ocurrencia de esa palabra para ese documento, como lo hicimos antes. Esto lo puedes hacer con el método `transform()` pasando el dataset como argumento. El método `tranform()`devuelve la matriz con enteros de NumPy, y esto lo puedes convertir a un arreglo utilizando `toarray()`.

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform

In [ ]:
doc_array = #TODO
doc_array

Ahora ya tienes una representación limpia de los documentos a partir de la distribución de frecuencias de las palabras. Para hacerlo más fácil de entender, vamos a convertir este arreglo a un dataframe y vamos a nombrar las columnas apropiadamente. Recuerda, tenemos la matriz en `doc_array`y los nombres de las columnas son los nombres de los features que vimos antes con `get_feature_names()`. Si no sabes cómo hacer esto, revisa los argumentos para construir el objeto DataFrame.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

In [ ]:
frequency_matrix = pd.DataFrame(#TODO)
frequency_matrix

¡Muy bien! Haz logrado implementar Bag of Words exitosamente para el dataset que creamos. Un problema potencial de utilizar este método es que nuestro dataset de texto puede ser muy largo (por ejemplo, si en vez de SMS usamos artículos o posts). El problema con esto es que habrán valores mucho más comunes por la estructura del lenguaje. Por lo tanto, esto podría sesgar los cálculos en la matriz y afectar el análisis. 

Hay varias técnicas para mitigar esto. Una manera es utilizar el método `stop_words` que mencionamos antes. Esto evitará tener una matriz esparcida. Otro método es utilizar el método tfidf. Ese método está fuera del alcance de este cuaderno (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer).  


## Paso 4.1. Training y testing sets
Ahora que ya sabemos cómo lidiar con el problema de Bag of Words, vamos a aplicarlo a nuestro dataset y proceder a analizarlo. El primer paso sería separar en training y testing para que podamos hacer el testing de nuestro modelo luego. En nuestro DataFrame tenemos dos columnas, `sms_message`, que es el input, y `label` que es el output que queremos predecir. Aquí hacemos la separación:

In [ ]:
# split into training and testing sets
# USE from sklearn.model_selection import train_test_split to avoid seeing deprecation warning.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'])

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

## Paso 4.2. Procesamiento con Bag of Words para nuestro dataset

Ahora que ya tenemos la información separada, nuestro objetivo es seguir los pasos de antes para implementar Bag of Words. Utilizaremos, como antes, `CountVectorizer()`.

Primero, haz el fit con nuestra información de entrenamiento (X_train) y devuelve la matriz.

Después, transforma tu testing data (X_test) para devolver la matriz.

¿Qué quiere decir esto?
* La matriz la diseñamos con la información de entrenamiento.
* La info de testing sólo se transforma. 

In [ ]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

## Paso 5.1 Implementación de Teorema de Bayes desde cero.

Ahora que el dataset ya está en el formato que necesitamos, podemos mover a la parte de crear un algoritmo que pueda predecir la clase de un mensaje (spam o no spam). Como mencionamos al inicio, el Teorema de Bayes calcula la probabilidad de que un evento ocurra basado en ciertas otrs probabilidades relacionadas con ese evento. Está compuesto por un prior (probabilidades que ya se conocen) y un posterior (probabilidades que queremos computar a partir de los prioris). 

Iniciaremos implementando Bayes desde cero utilizando un ejemplo sencillo. Digamos que queremos encontrar la probabilidad que un individuo tenga diabetes dado que él o ella se hizo una prueba y tiene un resultado positivo.


Asumimos lo siguiente:
* P(D) es la probabilidad de que una persona tenga diabetes. EL valor es 0.01, es decir, sólo el 1% de la población general tiene diabetes.
* P(Pos) es la probabilidad de que un resultado de una prueba sea positivo
* P(Neg) es la probabilidad de que un resultado de una prueba sea negativo.
* P(Pos|D) es la probabilidad de tener un resultado positivo en una prueba, dado que tienes diabetes. El valor es de 0.9. Es decir, la prueba está correcta 90% del tiempo. Esto se llama True Positive Rate o sensitividad.
* P(Neg|~D) es la probabilidad de tener un resultado negativo dado que no se tiene diabetes. Esto también tiene un valor de 0.9. Esto se llama especificidad o True Negative Rate.

![teorema de bayes](https://github.com/udacity/machine-learning/raw/85dfd1dba5ccfd5435f7fb19a6274b2f95a6bfb7/projects/practice_projects/naive_bayes_tutorial/images/bayes_formula.png "teorema de bayes")

* P(A) es la probabilidad a prior que un evento A ocurra independientemente. En nuestro ejemplo, es P(D).
* P(B) es la probabilidad a prior que un evento B ocurra independientemente. En nuestro ejemplo, es P(Pos).
* P(A|B) es la probabilidad posterior que un evento A ocurrado dado B. En nuestro ejemplo, esto sería P(D|Pos). Es decir, la probabilidad que un inidividuo tenga diabetes dado que tuvo un resultado positivo de la prueba. Este sería el valor que queremos calcular.
* P(B|A) es la probabilidad de que B ocurra dada A. En nuestro ejemplo es P(Pos|D). Este valor ya lo conocemos.

Ingresando nuestras probabilidad en la fórmula tenemos:

P(D|Pos) = (P(D) X P(Pos|D) / P(Pos)

La probabilidad de tener un resultado positivo P(Pos) puede ser calculada usando la especificidad y sensitividad:

P(Pos) = [P(D) X Sensitivity] + [P(~D) X (1-Specificity))]

In [ ]:
'''
Solution (skeleton code will be provided)
'''
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg/~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = # TODO
print('The probability of getting a positive test result P(Pos) is: {}',format(p_pos))

Podemos calcular los posteriores con esto de la siguiente manera:

P(D/Pos) = (P(D) x Sensitivity)) / P(Pos)

P(~D/Pos) = (P(~D) x (1-Specificity)) / P(Pos)

Las sumas de los posteriores siempre debe ser 1.

In [ ]:
# P(D|Pos)
p_diabetes_pos = # TODO
print('Probability of an individual having diabetes, given that that individual got a positive test result is:\
',format(p_diabetes_pos))

In [ ]:
# P(Pos/~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos = # TODO
print('Probability of an individual not having diabetes, given that that individual got a positive test result is:'\
,p_no_diabetes_pos)

¡Excelente! Implementaste el Teorema de Bayes. El análisis te debería mostrar que aunque el resultado de la prueba sea positivo, hay sólo un 8.3% de probabilidad que en realidad tenga diabetes. Esto es, claro, asumiendo que sólo el 1% de la población tiene diabetes. 

¿Por qué se llama Naive Bayes? ¿Qué significa 'Naive'?

El término viene del hecho que el algoritmo considera cada feature como si fuera independiente, lo cuál no siempre es el caso. En el ejemplo de diabetes, estamos considerando un sólo features, el resultado de la prueba. Digamos que agregamos un nuevo feature, 'ejercicio'. Este feature es un valor binario (0 ó 1), donde el 1 representa que el individuo hace ejercicio tres o más veces a la semana. Como estos features son dependientes, Bayes falla, pero Naive Bayes, una extensión del teorema, funciona bien por asumir que todos son independientes.

## Paso 5.2

Ya entendimos el Teorema de Bayes: ahora lo extenderemos para considerar más de un feature. Digamos que tenemos dos candidatos de partidos políticos, 'ANLO', del Partido Azul, y 'Anayo', del Partido Rojo. Digamos que tenemos la probabilidad de que los candidatos digan las palabras 'libertad', 'migración' y 'familia' cuando dan un discurso:

Probabilidad que ANLO diga 'libertad': P(L|A) = 0.1 

Probabilidad que ANLO diga 'migración': P(M|A) = 0.1 

Probabilidad que ANLO diga 'familia': P(F|A) = 0.8

Probabilidad que Anayo diga 'libertad': P(L|AY) = 0.7

Probabilidad que Anayo diga 'migración': P(M|AY) = 0.2 

Probabilidad que Anayo diga 'familia': P(F|AY) = 0.1

Ahora, asumamos que ambos tienen la misma probaiblidad de dar un discurso. P(A) y P(AY) = 0.5.

Dado esto, podemos encontrar la probabilidad de que el candidato sea ANLO dado que dijo las palabras 'libertad' y 'migración'. Aquí usamos el teorema de Bayes y usamos dos features (las dos palabras):

Given this, what if we had to find the probabilities of Jill Stein saying the words 'freedom' and 'immigration'? This is where the Naive Bayes'theorem comes into play as we are considering two features, 'freedom' and 'immigration'. Usaremos la siguiente fórmula:

![alt text](https://github.com/udacity/machine-learning/raw/85dfd1dba5ccfd5435f7fb19a6274b2f95a6bfb7/projects/practice_projects/naive_bayes_tutorial/images/naivebayes.png "texto")

Aquí, y es la clase (el nombre del candidato) y x1 a xn son los vectores con los features (palabras individuales). El teorema asume que cada feature es independiente de otro. Para esto, necesitamos calcular la siguiente probabilidad:

P(A|L, M): Probabilidad que sea ANLO dado que dijo libertad y migración.

Usando lo que sabemos de Bayes, podemos hacer P(A|L, M) = (P(A) * (P(L|A) * P(M|A))/P(L,M). Aquí, P(L,M) es la probabilidad de que las palabras 'libertad' y 'migración' sean dichas en un discurso. 

Iniciaremos calculando P(L,M).
* El primer paso es multiplicar la probabilidad de que Anlo de un discurso con sus probabilidades individuales para esas dos palabras.
* Repetiremos el paso pero con Anayo.
* Finalmente, sumamos ambas probabilidades. 

In [ ]:
# Paso 1

# P(A)
p_a = 0.5

# P(L/A)
p_a_l = 0.1

# P(M/A)
p_a_m = 0.1

# TODO
p_a_text =  #TODO
print(p_a_text)

In [ ]:
# Paso 2

# P(AY)
p_ay = 0.5

# P(L/AY)
p_ay_l = 0.7

# P(M/AY)
p_ay_m = 0.2

# TODO
p_ay_text = #TODO
print(p_ay_text)

In [ ]:
# Paso 3
p_l_m = #TODO
print('Probabilidad de palabras libertad y migración es: ', format(p_l_m))

Ahora podemos calcular la probabilidad de P(A|L,M) y P(AY|L,M). 

P(A|L, M) = (P(A) (P(L|A) P(M|A))/P(L,M)

In [ ]:
# TODO
p_a_lm = #todo
print('Probabilidad de ANLO diciendo libertas y migración: ', format(p_a_lm))

In [ ]:
# TODO
p_ay_lm = #TODO
print('Probabilidad de Anayo diciendo libertas y migración: ', format(p_ay_lm))

Como puedes ver, la suma de probabilidades posteriores debe ser igual a 1. Ya implementaste Naive Bayes desde cero. El análisis desmuestra que sólo hay un 6.6% de probabilidad que ANLO diga libertad y migración en su discurso en comparación.

Otro ejemplo de Naive Bayes es cuando buscamos 'Cruz Azul' en un search engine como Google. Para conseguir resultados que correspondan al equipo de futbol Cruz Azul, el search engine necesita asociar las dos palabras y no tratarlas individualmente.

Aplicándolo a nuestro problema de spam, Naive Bayes mirará cada palabra de manera independiente. Esto suele funcionar para spam ya que hay ciertas palabras que funcionan como banderas rojas y suelen garantizar que se clasifique el mensaje como spam. Por ejemplo, en los correos, el uso de la palabra viagra. 

## Paso 6. Naive Bayes con scikit-learn
Por suerte, scikit-learn tiene varias implementaciones de Naive Bayes que utilizaremos y no nos tendremos que preocupar con la matemáticas desde cero. Utilizaremos el método `sklearn.naive_bayes` para hacer predicciones en nuestro dataset. Específicamente, utilizaremos la implementación multinomial. Este clasificador en particular es bueno para features discretos. Utiliza los conteos de las palabras como entrada. En el otro lado, Gausian Naive Bayes funciona mejor para datos continuos ya que asume que la información sigue una distribución normal. 

Ya hemos cargado la información de entrenamiento en `training_data` y la de testing en `testing_data`. Usa `MultinomialNB` y haz un fit del training data en el clasificador utilizando `fit()`. Haz el fit con training_data y `y_train`

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# TODO
naive_bayes = #todo
naive_bayes.fit(#TODO)

Ahora que nuestro algoritmo ha sido entrenado con el training data, vamos a hacer predicciones con el testing set que está guardado en `testing_data`. Utiliza el método `predict()`.

In [ ]:
#TODO
predictions = naive_bayes.predict(#todo)

## Paso 7. Evaluación del modelo. 

Ahora que tenemos las predicciones con el test set, el siguiente objetivo es evaluar qué tan bueno fue nuestro modelo. Hay varios mecanismos que ya hemos visto.

* Accuracy: Mide qué tanto el clasificador hace predicciones correctas. Es la proporción de número de predicciones correctas con todas las predicciones.
* Precision: Mide qué proporción de los mensajes casificados como spam realmente fueron spam. Es la proporción de positivos verdaderos (palabras clasificadas como spam que eran spam) con todos los positivos (todas las palabras clasificadas como spam). Es decir, lo podemos calcular así:

[True Positives/(True Positives + False Positives)]

* Recall: O sensitividad. Mide qué proporción de mensajes que eran spam fueron clasificados como spam. Es la proporción de positivos verdaderos con todas las palabras que realmente eran spam. Es decir:

[True Positives/(True Positives + False Negatives)]

Este problema es de clasificación sesgada en su distribución. Esto quiere decir que de 100 mensajes, tal vez sólo 2 eran spam. Accuracy no es una buena métrica. Podríamos clasificar 90 mensajes como no spam, y aún así tendríamos tener buen accuracy. Por lo tanto, precision y recall nos sirve. Podemos combinar ambas métricas con F1, el cuál es un promedio de ambas. El valor va de 0 a 1, y 1 es el mejor.

Aún así, utilizaremos las 4 métricas para asegurarnos que nuestro modelo se desempeña bien.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(#TODO)
print('Precision score: ', format(#TODO)
print('Recall score: ', format(#TODO)
print('F1 score: ', format(#TODO)

## Paso 8.

Una de las mayores ventajas de Naive Bayes es su habilidad para manejar cantidades extremadamente grandes de features. En nuestro ejemplo, donde cada palabra es un feature, hay miles de features. También se desempeña bien cuando hay features que no importan y casi no es afectada por ellos. Adicionalmente, es una técnica relativamente simple, y suele funcionar sin ajustar sus hiperparámetros. Casi no tiene problemas de overfitting. ¡Se entrena muy rápido, también. Así que este es una excelente técnica!